In [2]:
import pandas as pd
import pickle
import os

In [4]:
master_df = pickle.load(open("./data/master_df.p", "rb"))

In [8]:
#creating random sample from master_df to label
sampled = master_df[master_df.problematic==1].sample(500)
sampled.to_excel("./data/fptp.xlsx")

In [5]:
#merge by row index
labeled = pd.read_excel("./data/fptp_labeled.xlsx",index_col=0)
master_df_labeled = master_df.join(labeled.true_pos).fillna(-1)


In [32]:
master_df_labeled[master_df_labeled.true_pos==0].shape

(40, 5)

In [34]:
master_df_labeled[master_df_labeled.true_pos==1].shape

(42, 5)

In [6]:
pickle.dump(master_df_labeled,open("./data/master_df_labeled.p","wb"))